In [ ]:
import simpy
import numpy as np
import matplotlib.pyplot as plt

class AirportSimulation:
    def __init__(self, env, num_security_personnel, num_ticket_checkers, num_gates, gate_capacity, baggage_capacity, num_passengers, arrival_rate):
        self.env = env
        self.num_security_personnel = num_security_personnel
        self.num_ticket_checkers = num_ticket_checkers
        self.num_gates = num_gates
        self.gate_capacity = gate_capacity
        self.num_passengers = num_passengers
        self.arrival_rate = arrival_rate
        self.security = simpy.Resource(env, capacity=num_security_personnel)
        self.ticket_check = simpy.Resource(env, capacity=num_ticket_checkers)
        self.gates = [simpy.Resource(env, capacity=gate_capacity) for _ in range(num_gates)]
        self.baggage_container = simpy.Container(env, init=0, capacity=baggage_capacity)
        self.security_wait_times = []
        self.ticket_check_wait_times = []
        self.baggage_check_wait_times = []
        self.boarding_wait_times = []

    def passenger_flow(self):
        for i in range(self.num_passengers):
            arrival_time = np.random.exponential(1 / self.arrival_rate)
            yield self.env.timeout(arrival_time)
            self.env.process(self.passenger(i))

    def passenger(self, passenger_id):
        arrival_time = self.env.now
        # Security Check
        security_start_time = self.env.now
        with self.security.request() as req:
            yield req
            yield self.env.timeout(np.random.triangular(1, 3, 5))
            security_end_time = self.env.now
            self.security_wait_times.append(security_end_time - security_start_time)

        # Ticket Check
        ticket_check_start_time = self.env.now
        with self.ticket_check.request() as req:
            yield req
            yield self.env.timeout(np.random.triangular(1, 2, 4))
            ticket_check_end_time = self.env.now
            self.ticket_check_wait_times.append(ticket_check_end_time - ticket_check_start_time)

        # Baggage Check
        baggage_check_start_time = self.env.now
        yield self.baggage_container.put(1)
        yield self.env.timeout(np.random.triangular(1, 3, 5))
        baggage_check_end_time = self.env.now
        self.baggage_check_wait_times.append(baggage_check_end_time - baggage_check_start_time)
        
        # Boarding the Plane
        boarding_start_time = self.env.now
        gate_number = passenger_id % self.num_gates
        request = self.gates[gate_number].request()
        yield request
        yield self.env.timeout(np.random.triangular(5, 10, 15))
        boarding_end_time = self.env.now
        self.boarding_wait_times.append(boarding_end_time - boarding_start_time)
        self.gates[gate_number].release(request)

def run_simulation(num_security_personnel, num_ticket_checkers):
    env = simpy.Environment()
    airport_sim = AirportSimulation(env, num_security_personnel, num_ticket_checkers, num_gates, gate_capacity, baggage_capacity, num_passengers, arrival_rate)
    env.process(airport_sim.passenger_flow())
    env.run()
    avg_security_wait = np.mean(airport_sim.security_wait_times)
    avg_ticket_check_wait = np.mean(airport_sim.ticket_check_wait_times)
    avg_baggage_check_wait = np.mean(airport_sim.baggage_check_wait_times)
    avg_boarding_wait = np.mean(airport_sim.boarding_wait_times)
    avg_total_wait = (avg_security_wait + avg_ticket_check_wait + avg_baggage_check_wait + avg_boarding_wait) / 4
    return avg_total_wait


num_gates = 3
gate_capacity = 50
baggage_capacity = 100
num_passengers = 100
arrival_rate = 1


security_personnel_salary = 5000  
ticket_checker_salary = 4500     

results = []

for security_personnel in range(1, 6):
    for ticket_checkers in range(1, 6):
        avg_wait_time = run_simulation(security_personnel, ticket_checkers)
        total_cost = (security_personnel * security_personnel_salary) + (ticket_checkers * ticket_checker_salary)
        weighted_score = (0.5 * avg_wait_time) + (0.5 * total_cost / 10000)  # Örnek ağırlıklar
        results.append((security_personnel, ticket_checkers, avg_wait_time, total_cost, weighted_score))
        print(f"Güvenlik personeli: {security_personnel}, Bilet kontrol personeli: {ticket_checkers}, Ortalama bekleme süresi: {avg_wait_time:.2f} dakika, Toplam maliyet: {total_cost}, Ağırlıklı skor: {weighted_score:.2f}")


optimal_result = min(results, key=lambda x: x[4])

print(f"\nEn optimum sonuç: Güvenlik personeli: {optimal_result[0]}, Bilet kontrol personeli: {optimal_result[1]}, Ortalama bekleme süresi: {optimal_result[2]:.2f} dakika, Toplam maliyet: {optimal_result[3]}, Ağırlıklı skor: {optimal_result[4]:.2f}")


security_personnel_vals = [result[0] for result in results]
ticket_checkers_vals = [result[1] for result in results]
avg_wait_times = [result[2] for result in results]
total_costs = [result[3] for result in results]
weighted_scores = [result[4] for result in results]

fig = plt.figure(figsize=(15, 6))

ax = fig.add_subplot(131, projection='3d')
ax.scatter(security_personnel_vals, ticket_checkers_vals, avg_wait_times, c='r', marker='o')
ax.set_xlabel('Güvenlik Personeli Sayısı')
ax.set_ylabel('Bilet Kontrol Personeli Sayısı')
ax.set_zlabel('Ortalama Bekleme Süresi (dakika)')
ax.set_title('Bekleme Süreleri')

ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(security_personnel_vals, ticket_checkers_vals, total_costs, c='b', marker='o')
ax2.set_xlabel('Güvenlik Personeli Sayısı')
ax2.set_ylabel('Bilet Kontrol Personeli Sayısı')
ax2.set_zlabel('Toplam Maliyet')
ax2.set_title('Toplam Maliyet')

ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(security_personnel_vals, ticket_checkers_vals, weighted_scores, c='g', marker='o')
ax3.set_xlabel('Güvenlik Personeli Sayısı')
ax3.set_ylabel('Bilet Kontrol Personeli Sayısı')
ax3.set_zlabel('Ağırlıklı Skor')
ax3.set_title('Ağırlıklı Skor')

plt.suptitle('Personel Sayısına Göre Bekleme Süreleri ve Maliyet Analizi')
plt.tight_layout()
plt.show()
